In [55]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.utils import resample
from tqdm import tqdm
import umap
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class ProgressBar:
    def __init__(self, total_epochs):
        self.pbar = tqdm(total=total_epochs, desc="UMAP Training Progress", unit="epoch")

    def __call__(self, epoch, n_epochs):
        self.pbar.n = epoch
        self.pbar.refresh()

n_epochs = 200
progress_bar = ProgressBar(total_epochs=n_epochs)

UMAP Training Progress:   0%|          | 0/200 [10:49<?, ?epoch/s]


In [ ]:
all = pd.read_parquet(r"D:\Pythonowe rzeczki\Pliki genowe\plikp.parquet")
# data = all[all["go_term"].str.contains("|".join(["mitochondria","chaperon","phagy","transport","regulation"]),case=False,na=False)]
data = all
descriptions = data['go_term']

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', max_features=1000)
X = tfidf.fit_transform(descriptions)

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X)
data['Cluster'] = clusters

In [53]:
all["go_term"]

0                    cell surface receptor signaling pathway
1          positive regulation of I-kappaB kinase/NF-kapp...
2          negative regulation of viral-induced cytoplasm...
3             negative regulation of NIK/NF-kappaB signaling
4                                        signal transduction
                                 ...                        
1048570                               nucleic acid transport
1048571                                        RNA transport
1048572                              regulation of transport
1048573                     positive regulation of transport
1048574                                       nuclear export
Name: go_term, Length: 1048575, dtype: object

In [ ]:
X_sampled = resample(X, n_samples=10000, random_state=42)

reducer = umap.UMAP(
    n_components=2,
    random_state=42,
    n_neighbors=5,
    min_dist=0.1,
    n_epochs=100,
    low_memory=True 
)

X_embedded = reducer.fit_transform(X_sampled)

data['UMAP1'] = X_embedded[:, 0]
data['UMAP2'] = X_embedded[:, 1]

plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, x='UMAP1', y='UMAP2', hue='Cluster', palette='viridis', s=60)

for i in range(len(data)):
    plt.text(data.iloc[i]['UMAP1'], data.iloc[i]['UMAP2'], str(data.iloc[i]['Cluster']), fontsize=8, alpha=0.7)

plt.title("Wizualizacja klastrów GO za pomocą UMAP")
plt.show()

d:\Pythonowe rzeczki\Pliki genowe\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
d:\Pythonowe rzeczki\Pliki genowe\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
d:\Pythonowe rzeczki\Pliki genowe\.venv\Lib\site-packages\umap\spectral.py:548: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(
d:\Pythonowe rzeczki\Pliki genowe\.venv\Lib\site-packages\umap\spectral.py:548: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(
d:\

ValueError: Length of values (10000) does not match length of index (1048575)

In [27]:
data[data["UMAP1"]>10].sort_values(by="UMAP1")

,gene_id,go_id,go_ontology,go_term,Cluster,UMAP1,UMAP2
1012397,ENSG00000137992,GO:0005947,CC,mitochondrial alpha-ketoglutarate dehydrogenas...,0,10.064670,-6.582705
911383,ENSG00000134056,GO:0005947,CC,mitochondrial alpha-ketoglutarate dehydrogenas...,0,10.064754,-6.582644
397664,ENSG00000103507,GO:0005947,CC,mitochondrial alpha-ketoglutarate dehydrogenas...,0,10.064919,-6.582370
229645,ENSG00000083123,GO:0005947,CC,mitochondrial alpha-ketoglutarate dehydrogenas...,0,10.065327,-6.582176
15645,ENSG00000005882,GO:0005967,CC,mitochondrial pyruvate dehydrogenase complex,0,10.067005,-6.581408
...,...,...,...,...,...,...,...
113880,ENSG00000060762,GO:0031304,CC,intrinsic component of mitochondrial inner mem...,3,24.989958,-2.321104
27409,ENSG00000007923,GO:0031304,CC,intrinsic component of mitochondrial inner mem...,3,24.989990,-2.321098
252513,ENSG00000087299,GO:0031304,CC,intrinsic component of mitochondrial inner mem...,3,24.989990,-2.321035
846950,ENSG00000130489,GO:0031304,CC,intrinsic component of mitochondrial inner mem...,3,24.990030,-2.321081
